In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import matplotlib.pyplot as plt


from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

In [ ]:
input_shape = (224,224)
batch_size = 32

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=input_shape,
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=input_shape,
  batch_size=batch_size)

In [ ]:
num_classes = 5
input_shape = (224,224,3)

model_input = keras.Input(shape=(224, 224,3), name="f_input")
x = layers.Conv2D(16, 3,padding="same", activation="relu")(model_input)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(32, 3,padding="same", activation="relu")(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3,padding="same", activation="relu")(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(128,activation="relu")(x)
model_output = layers.Dense(num_classes,activation="softmax")(x)

functional_model = keras.Model(model_input, model_output, name="functional_model")

In [ ]:
functional_model.summary()

In [ ]:
functional_model.compile(optimizer='Adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['acc'])

In [ ]:
epochs=10
history = functional_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
num_classes = 5
input_shape = (224,224,3)

def func_model(num_classes, input_shape):
    model_input = keras.Input(shape=input_shape, name="f_input")
    x = layers.Conv2D(16, 3,padding="same", activation="relu")(model_input)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(32, 3,padding="same", activation="relu")(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3,padding="same", activation="relu")(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128,activation="relu")(x)
    model_output = layers.Dense(num_classes,activation="softmax")(x)

    return keras.Model(model_input, model_output, name="functional_model")

In [ ]:
functional_model = func_model(num_classes,input_shape)

In [ ]:
functional_model.summary()

In [ ]:
class MyModel(tf.keras.Model):

  def __init__(self,num_classes,input_shape):
    super().__init__()
    self.conv1 = layers.Conv2D(16, 3,padding="same", activation="relu",input_shape=input_shape)
    self.conv2 = layers.Conv2D(32, 3,padding="same", activation="relu")
    self.conv3 = layers.Conv2D(64, 3,padding="same", activation="relu")
    self.maxpool2d = layers.MaxPooling2D()
    self.flatten = layers.Flatten()
    self.dense1 = layers.Dense(128,activation="relu")
    self.dense2 = layers.Dense(num_classes,activation="softmax")
    

  def call(self, inputs):
    x = self.conv1(inputs)
    x = self.maxpool2d(x)
    x = self.conv2(x)
    x = self.maxpool2d(x)
    x = self.conv3(x)
    x = self.maxpool2d(x)

    x = self.flatten(x)
    x = self.dense1(x)
    out = self.dense2(x)

    return out

subclass_model = MyModel(num_classes,input_shape)

In [ ]:
subclass_model.compile(optimizer='Adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['acc'])

In [ ]:
epochs=10
history = subclass_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
num_classes = 5
input_shape = (224,224,3)

model_input = keras.Input(shape=(224, 224,3), name="f_input")
x1 = layers.Conv2D(16, 3,padding="same", activation="relu")(model_input)
x1 = layers.MaxPooling2D()(x1)
x2 = layers.Conv2D(32, 3,padding="same", activation="relu")(x1)
x2 = layers.MaxPooling2D()(x2)
x3 = layers.Conv2D(64, 3,padding="same", activation="relu")(x1)
x3 = layers.MaxPooling2D()(x3)
flat2 = layers.Flatten()(x2)
flat3 = layers.Flatten()(x3)
d1 = layers.Dense(128,activation="relu")(flat2)
d2 = layers.Dense(num_classes,activation="softmax")(flat3)

functional_model = keras.Model(model_input, (d1,d2))

In [ ]:
functional_model.summary()

In [ ]:
num_classes = 5
input_shape = (224,224,3)

model_input = keras.Input(shape=(224, 224,3), name="f_input")
x1 = layers.Conv2D(16, 3,padding="same", activation="relu")(model_input)
x1 = layers.MaxPooling2D()(x1)
x2 = layers.Conv2D(32, 3,padding="same", activation="relu")(x1)
x2 = layers.MaxPooling2D()(x2)
x3 = layers.Conv2D(64, 3,padding="same", activation="relu")(x1)
x3 = layers.MaxPooling2D()(x3)

con1 = layers.Concatenate()([x2,x3])
flat = layers.Flatten()(con1)

d1 = layers.Dense(128,activation="relu")(flat)
d2 = layers.Dense(num_classes,activation="softmax")(d1)

functional_model = keras.Model(model_input, d2)

In [ ]:
functional_model.summary()

In [ ]:
functional_model.summary()

<h1>GoogLeNet</h1>


<h3>201702052 이동헌</h3>

In [38]:
from tensorflow import keras
from tensorflow.keras import layers

'''
GoogLeNet 코드 작성
functional API, subclass model 중 원하는 방식으로 구현
함수를 사용하면 보다 간편하게 구현 가능
'''

def inception_module(input_layer,filter1,filter2,filter3):
  for_con0 = layers.Conv2D(filters=filter1, kernel_size=(1,1), strides=(1,1), padding='same', activation='relu')(input_layer)

  for_con1 = layers.Conv2D(filters=filter1, kernel_size=(1,1), strides=(1,1), padding='same', activation='relu')(input_layer)
  for_con1 = layers.Conv2D(filters=filter2, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(for_con1)

  for_con2 = layers.Conv2D(filters=filter1, kernel_size=(1,1), strides=(1,1), padding='same', activation='relu')(input_layer)
  for_con2 = layers.Conv2D(filters=filter3, kernel_size=(5,5), strides=(1,1), padding='same', activation='relu')(for_con2)

  for_con3 = layers.MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same')(input_layer)
  for_con3 = layers.Conv2D(filters=filter1, kernel_size=(1,1), strides=(1,1), padding='same', activation='relu')(for_con3)

  outputs = layers.Concatenate()([for_con0, for_con1, for_con2, for_con3])
  return outputs

In [37]:
model_input = keras.Input(shape=(224, 224,3), name="f_input")

x = layers.Conv2D(filters=64, kernel_size=(7,7), strides=(2,2), padding='same', activation='relu')(model_input)
x = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)
x = layers.LayerNormalization()(x)

x = layers.Conv2D(filters=64, kernel_size=(1,1), strides=(1,1), padding='same', activation='relu')(x)
x = layers.Conv2D(filters=192, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(x)
x = layers.LayerNormalization()(x)
x = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)

x = inception_module(x, 64, 128, 32)
x = inception_module(x, 128, 192, 96)
x = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)

x = inception_module(x, 192, 208, 48)
x = inception_module(x, 160, 224, 64)
x = inception_module(x, 128, 256, 64)
x = inception_module(x, 112, 288, 64)
x = inception_module(x, 256, 320, 128)
x = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)

x = inception_module(x, 256, 320, 128)
x = inception_module(x, 384, 384, 128)

x = layers.AveragePooling2D(pool_size=(7,7), strides=(1,1), padding='same')(x)
x = layers.Dropout(0.4)(x)

x = layers.Dense(1000, activation='softmax')(x)
model_output = layers.Dense(5, activation="softmax")(x)

functional_model = keras.Model(model_input, model_output, name="GoogLeNet")

functional_model.summary()


Model: "GoogLeNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 f_input (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_248 (Conv2D)            (None, 112, 112, 64  9472        ['f_input[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d_75 (MaxPooling2D  (None, 56, 56, 64)  0           ['conv2d_248[0][0]']             
 )                                                                                        